In [ ]:
import numpy as np
import pandas as pd #To convert txt to dataframe so that other things will be easier
from tqdm import tqdm #For progress checking while running the code
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score  #For metrices calculation
import warnings
warnings.filterwarnings("ignore")

In [ ]:
train= open("/content/drive/MyDrive/NLP_ass2/NLP2/train.txt",'r')
test=open("/content/drive/MyDrive/NLP_ass2/NLP2/test.txt",'r')

Change train txt file to a dataframe

In [ ]:
id= None
text = None
df_train=pd.DataFrame(columns=['sent_id','text','word', 'normalized_word', 'POS_tag', 'head_word_index','dependency_relation' ])
for line in train:
  if line.strip():
    if line.startswith("# sent_id"):
        id = line.split("=")[-1].strip()
    elif line.startswith("# text"):
         text = line.split("=")[-1].strip()
    else:
      parts = line.split()
      row = {
                'sent_id': id,
                'text': text,
                'word': parts[1],
                'normalized_word': parts[2],
                'POS_tag': parts[3],
                'head_word_index': parts[4],
                'dependency_relation': parts[5]
            }
      df_train = df_train.append(row, ignore_index=True)

In [ ]:
df_train

,sent_id,text,word,normalized_word,POS_tag,head_word_index,dependency_relation
0,GUM_academic_art-1,Aesthetic Appreciation and Spanish Art:,Aesthetic,aesthetic,JJ,2,amod
1,GUM_academic_art-1,Aesthetic Appreciation and Spanish Art:,Appreciation,appreciation,NN,0,root
2,GUM_academic_art-1,Aesthetic Appreciation and Spanish Art:,and,and,CC,5,cc
3,GUM_academic_art-1,Aesthetic Appreciation and Spanish Art:,Spanish,Spanish,JJ,5,amod
4,GUM_academic_art-1,Aesthetic Appreciation and Spanish Art:,Art,art,NN,2,conj
...,...,...,...,...,...,...,...
36707,GUM_conversation_gossip-26,That doesn't matter or?,matter,matter,VB,0,root
36708,GUM_conversation_gossip-26,That doesn't matter or?,or,or,CC,4,conj
36709,GUM_conversation_gossip-26,That doesn't matter or?,?,?,.,4,punct
36710,GUM_conversation_gossip-27,Mmm.,Mmm,mmm,UH,0,root


Change test txt file to a dataframe

In [ ]:
id= None
text = None
df_test=pd.DataFrame(columns=['sent_id','text','word', 'normalized_word', 'POS_tag', 'head_word_index','dependency_relation' ])
for line in test:
  if line.strip():
    if line.startswith("# sent_id"):
        id = line.split("=")[-1].strip()
    elif line.startswith("# text"):
         text = line.split("=")[-1].strip()
    else:
      parts = line.split()
      row = {
                'sent_id': id,
                'text': text,
                'word': parts[1],
                'normalized_word': parts[2],
                'POS_tag': parts[3],
                'head_word_index': parts[4],
                'dependency_relation': parts[5]
            }
      df_test = df_test.append(row, ignore_index=True)

In [ ]:
df_test.shape

(2184, 7)

Creating a dictionary to store frequency of each word

In [ ]:
word_freq= df_train['word'].value_counts()

print(len(word_freq))
# (word_freq[','])

6782


Calculating how much tokens in train data are also present in test data

In [ ]:
test_token= df_test['word'].tolist()
yes=0
no=0
for item in test_token:
  if item in word_freq:
      yes +=1
  else :
    no +=1
print(f"present: {yes}, absent : {no}")

present: 1724, absent : 460


Unique POS tags and their count in the training dataset

In [ ]:
tags_set=df_train['POS_tag'].unique()
print(tags_set)
print(len(tags_set))

['JJ' 'NN' 'CC' ':' 'NNS' 'IN' 'HYPH' 'NNP' ',' 'WRB' 'VBP' 'VB' '.' 'WDT'
 'PRP' 'DT' 'VBZ' 'VBG' 'RB' 'MD' 'PRP$' 'VBN' 'FW' 'SYM' 'VBD' 'TO' 'RP'
 '-LRB-' '-RRB-' '_' 'POS' 'CD' 'EX' 'LS' 'JJR' 'RBR' 'WP$' 'JJS' 'PDT'
 '``' "''" 'WP' 'NNPS' 'UH' 'RBS' 'GW' '$']
47


Frequency of each tag

In [ ]:
tag_freq=df_train['POS_tag'].value_counts()
print(tag_freq.sum())
print(len(tag_freq))

36712
47


In [ ]:
tag_freq['DT']

2971

Frequency of each tag to appear in start of sentence

In [ ]:
df_train['start_of_sentence'] = df_train['sent_id'] != df_train['sent_id'].shift()
tag_start_freq= df_train[df_train['start_of_sentence']]['POS_tag'].value_counts()

Assigning 0 frequncy to those tags which doesnt appear at the start of sentence, so that tag_start_freq will store frequency of all 47 tags to come at the start of sentence.

In [ ]:
all_tags = df_train['POS_tag'].unique()
tag_start_freq = tag_start_freq.reindex(all_tags, fill_value=0)

In [ ]:
tag_start_freq

JJ        77
NN        61
CC       104
:          0
NNS       22
IN       282
HYPH       0
NNP      188
,          0
WRB       19
VBP       14
VB        23
.          0
WDT        6
PRP      283
DT       197
VBZ       13
VBG       16
RB       166
MD         3
PRP$      54
VBN       13
FW         6
SYM        0
VBD        5
TO         7
RP         0
-LRB-      1
-RRB-      0
_         85
POS        0
CD        38
EX         9
LS        15
JJR        2
RBR        6
WP$        0
JJS        3
PDT        4
``         5
''         0
WP        15
NNPS       2
UH       254
RBS        2
GW         0
$          0
Name: POS_tag, dtype: int64

Calculating count of each tag-tag pair in a dataset

In [ ]:
tag_pair_counts = {}
count = 0

for sent_id, group in df_train.groupby('sent_id'):
    tag_pairs = list(zip(group['POS_tag'], group['POS_tag'].shift(-1)))
    # Remove the last tag pair as it is a pair of tag of last element of sentence and None
    tag_pairs.pop()

    # print(f"sentence id: {sent_id}, tag pair: {tag_pairs}")
    for tag_pair in tag_pairs:
        if tag_pair not in tag_pair_counts:
            tag_pair_counts[tag_pair] = 1
        else:
            tag_pair_counts[tag_pair] += 1
    count += 1

# No. of sentencec processed
print(count)

2000


In [ ]:
tag_pair_counts['DT','NN']

1326

Calculating the count of each word-tag pair in a dataset

In [ ]:
df_train['word_tag_pair'] = list(zip(df_train['word'], df_train['POS_tag']))
tag_word_pair_counts= df_train['word_tag_pair'].value_counts()

In [ ]:
tag_word_pair_counts

(,, ,)               2207
(., .)               1615
(the, DT)            1560
(of, IN)             1024
(and, CC)             804
                     ... 
(Continuity, NNP)       1
(1918, CD)              1
(Movement, NNP)         1
(Cornell, NNP)          1
(Mmm, UH)               1
Name: word_tag_pair, Length: 7376, dtype: int64

len of tag_word_par_counts is more than no. of unique words in the dataset, as there are some words which have different tags in different context of sentences

In [ ]:
len(tag_word_pair_counts)

7376

In [ ]:
def tag_start_prob(t):
  if t in tag_start_freq:
      return tag_start_freq[t]/ 2000
  else:
      return 0

This function will give transition probability from tag1(t1) to tag2(t2)

P(t2 | t1) = Count( t1 , t2) / Count (t1)

In [ ]:
def transition_prob(t1, t2):
  if (t1,t2) in tag_pair_counts:
      return tag_pair_counts[(t1,t2)]/ tag_freq[t1]
  else:
      return 0

This function will give emission probability from tag(t) to word(w)

P(w |t ) = Count( t, w) / Count (t)

In [ ]:
def emission_prob(w,t):
  if (w,t) in tag_word_pair_counts:
      return tag_word_pair_counts[(w,t)]/ tag_freq[t]
  else :
      return 0

In [ ]:
tag_start_prob('JJ')

0.0385

In [ ]:
# transition_prob('DT','NN')
transition_prob('DT','CC')

0.0010097610232245036

In [ ]:
emission_prob('the','DT')
# emission_prob('and','CC')

0.5250757320767419

Creating transition matrix of tags

transition_matrix[i,j] will show the probability of transition from tag i to tag j

In [ ]:
T=len(tags_set)
transition_matrix=np.zeros((T,T))
for i in range(T):
  for j in range(T):
      transition_matrix[i,j]= transition_prob(tags_set[i], tags_set[j])

In [ ]:
transition_matrix.shape

(47, 47)

In [ ]:
transition_matrix[np.where(tags_set== 'DT')[0][0]][np.where(tags_set == 'NN')[0][0]]

0.4463143722652306

In [ ]:
def viterbi_train(sentence, tags_set):
    T=len(tags_set)
    W= len(sentence)
    epsilon = 1

    # Initialize the dynamic programming table to scores and tags found at each step
    scores= np.zeros((T,W), dtype= float)
    backptr = np.zeros((T,W), dtype=int)

    # Initialization Step for w==0 i,e. 1st word of sentence
    for t in range(T):
          scores[t,0]= (tag_start_prob(tags_set[t])* emission_prob(sentence[0], tags_set[t]))
          backptr[t, 0] = 0

    # Iteration Step
    for w in range (1,W):
      for t in range(T):
        prob= scores[:, w-1] * transition_matrix[:, t] * emission_prob(sentence[w], tags_set[t])

        backptr[t, w] = np.argmax(prob)
        scores[t, w] = np.max(prob)

     # Identifying final best path probability
    best_path_prob = np.max(scores[:, -1])

    # Path backtracking
    best_path = np.zeros(len(sentence), dtype=int)
    best_path[-1] = np.argmax(scores[:, -1])
    for w in range(len(sentence)-2, -1, -1):
        best_path[w] = backptr[best_path[w+1], w+1]

    POS_tag_seq = [tags_set[index] for index in best_path]
    return POS_tag_seq

In [ ]:
# Initialize lists to store train output
output_train = {
    'sent_id': [],
    'token_no': [],
    'word': [],
    'predicted_tag': []
}
POS_tags_train_pred=[]
POS_tags_train_true=[]
# Iterate over each sentence in the DataFrame
for sent_id, group in tqdm(df_train.groupby('sent_id')):
    sentence = group['word'].tolist()
    true_tag = group['POS_tag'].tolist()
    pred_tag = viterbi_train(sentence, tags_set)
    POS_tags_train_pred.extend(pred_tag)
    POS_tags_train_true.extend(true_tag)
    print(f"Sentence ID: {sent_id}, Words: {sentence}, POS Tag true : {true_tag}, POS Tag predicted: {pred_tag}")

    # Store the train output for each word in the sentence
    for i, (word, true_tag, pred_tag) in enumerate(zip(sentence, true_tag, pred_tag), start=1):
        output_train['sent_id'].append(sent_id)
        output_train['token_no'].append(i)
        output_train['word'].append(word)
        output_train['predicted_tag'].append(pred_tag)

  0%|          | 5/2000 [00:00<00:51, 38.46it/s]

Sentence ID: GUM_academic_art-1, Words: ['Aesthetic', 'Appreciation', 'and', 'Spanish', 'Art', ':'], POS Tag true : ['JJ', 'NN', 'CC', 'JJ', 'NN', ':'], POS Tag predicted: ['JJ', 'NN', 'CC', 'NNP', 'NNP', ':']
Sentence ID: GUM_academic_art-10, Words: ['The', 'viewing', 'experience', 'of', 'art', 'is', 'a', 'complex', 'one', ',', 'involving', 'issues', 'of', 'perception', ',', 'attention', ',', 'memory', ',', 'decision', '-', 'making', ',', 'affect', ',', 'and', 'emotion', '.'], POS Tag true : ['DT', 'NN', 'NN', 'IN', 'NN', 'VBZ', 'DT', 'JJ', 'NN', ',', 'VBG', 'NNS', 'IN', 'NN', ',', 'NN', ',', 'NN', ',', 'NN', 'HYPH', 'VBG', ',', 'NN', ',', 'CC', 'NN', '.'], POS Tag predicted: ['DT', 'NN', 'NN', 'IN', 'NN', 'VBZ', 'DT', 'JJ', 'NN', ',', 'VBG', 'NNS', 'IN', 'NN', ',', 'NN', ',', 'NN', ',', 'NN', 'HYPH', 'VBG', ',', 'VBP', ',', 'CC', 'NN', '.']
Sentence ID: GUM_academic_art-11, Words: ['Thus', ',', 'the', 'time', 'it', 'takes', 'and', 'the', 'ways', 'of', 'visually', 'exploring', 'an', '

  1%|          | 13/2000 [00:00<00:54, 36.76it/s]

Sentence ID: GUM_academic_art-17, Words: ['We', 'will', 'discuss', 'the', 'potential', 'implications', 'of', 'these', 'techniques', 'and', 'our', 'understanding', 'of', 'visual', 'behaviours', 'on', 'museum', 'and', 'gallery', 'practice', '.'], POS Tag true : ['PRP', 'MD', 'VB', 'DT', 'JJ', 'NNS', 'IN', 'DT', 'NNS', 'CC', 'PRP$', 'NN', 'IN', 'JJ', 'NNS', 'IN', 'NN', 'CC', 'NN', 'NN', '.'], POS Tag predicted: ['PRP', 'MD', 'VB', 'DT', 'JJ', 'NNS', 'IN', 'DT', 'NNS', 'CC', 'PRP$', 'NN', 'IN', 'JJ', 'NNS', 'IN', 'NN', 'CC', 'NN', 'NN', '.']
Sentence ID: GUM_academic_art-18, Words: ['The', 'project', 'brings', 'together', 'established', 'research', 'strengths', 'in', 'Spanish', 'art', 'history', ',', 'experimental', 'psychology', ',', 'digital', 'humanities', ',', 'and', 'museum', 'studies', 'to', 'explore', ',', 'using', 'eye', '-', 'tracking', 'techniques', ',', 'aesthetic', 'reactions', 'to', 'digital', 'representations', 'of', 'the', 'individual', 'Zurbarán', 'artworks', 'as', 'well', 

  1%|          | 17/2000 [00:00<01:02, 31.93it/s]

...

Sentence ID: GUM_conversation_family-70, Words: ['Cause', "that's", 'that', "'s", 'what', 'she', 'does', 'with', 'other', 'people', '.'], POS Tag true : ['IN', '_', 'DT', 'VBZ', 'WP', 'PRP', 'VBZ', 'IN', 'JJ', 'NNS', '.'], POS Tag predicted: ['IN', '_', 'WDT', 'VBZ', 'WP', 'PRP', 'VBZ', 'IN', 'JJ', 'NNS', '.']
Sentence ID: GUM_conversation_family-71, Words: ['Mm', '.'], POS Tag true : ['UH', '.'], POS Tag predicted: ['UH', '.']
Sentence ID: GUM_conversation_family-72, Words: ['But', 'all', 'winter', 'she', 'fixes', 'him', 'lunch', '.'], POS Tag true : ['CC', 'DT', 'NN', 'PRP', 'VBZ', 'PRP', 'NN', '.'], POS Tag predicted: ['CC', 'DT', 'NN', 'PRP', 'VBZ', 'PRP', 'NN', '.']
Sentence ID: GUM_conversation_family-73, Words: ['Oh', ',', 'cause', "they're", 'they', "'re", 'always', 'close', 'enough', 'to', '–'], POS Tag true : ['UH', ',', 'IN', '_', 'PRP', 'VBP', 'RB', 'RB', 'RB', 'TO', ':'], POS Tag predicted: ['UH', ',', 'IN', '_', 'PRP', 'VBP', 'RB', 'RB', 'RB', 'TO', ':']
Sentence ID: GUM_

 99%|█████████▉| 1975/2000 [00:51<00:00, 97.21it/s]

Sentence ID: GUM_conversation_family-88, Words: ['I', "don't", 'do', "n't", 'even', 'like', 'ice', 'tea', '.'], POS Tag true : ['PRP', '_', 'VBP', 'RB', 'RB', 'VB', 'NN', 'NN', '.'], POS Tag predicted: ['PRP', '_', 'VBP', 'RB', 'RB', 'JJ', 'NN', 'NN', '.']
Sentence ID: GUM_conversation_family-89, Words: ['Do', 'you', 'like', 'hot', 'tea', '?'], POS Tag true : ['VBP', 'PRP', 'VB', 'JJ', 'NN', '.'], POS Tag predicted: ['VB', 'PRP', 'VBP', 'JJ', 'NN', '.']
Sentence ID: GUM_conversation_family-9, Words: ['As', 'long', 'as', 'she', "didn't", 'did', "n't", 'get', 'up', '.'], POS Tag true : ['RB', 'RB', 'IN', 'PRP', '_', 'VBD', 'RB', 'VB', 'RB', '.'], POS Tag predicted: ['IN', 'JJ', 'IN', 'PRP', '_', 'VBD', 'RB', 'VB', 'RP', '.']
Sentence ID: GUM_conversation_family-90, Words: ['Yeah', ',', 'I', 'love', 'hot', 'tea', '.'], POS Tag true : ['UH', ',', 'PRP', 'VBP', 'JJ', 'NN', '.'], POS Tag predicted: ['UH', ',', 'PRP', 'VBP', 'JJ', 'NN', '.']
Sentence ID: GUM_conversation_family-91, Words: ['B

100%|██████████| 2000/2000 [00:51<00:00, 38.56it/s]

Sentence ID: GUM_conversation_gossip-2, Words: ['You', 'should', 'have', 'heard', 'him', ',', 'the', 'night', 'we', 'took', 'him', 'to', 'the', 'emergency', 'room', '.'], POS Tag true : ['PRP', 'MD', 'VB', 'VBN', 'PRP', ',', 'DT', 'NN', 'PRP', 'VBD', 'PRP', 'IN', 'DT', 'NN', 'NN', '.'], POS Tag predicted: ['PRP', 'MD', 'VB', 'VBN', 'PRP', ',', 'DT', 'NN', 'PRP', 'VBD', 'PRP', 'IN', 'DT', 'NN', 'NN', '.']
Sentence ID: GUM_conversation_gossip-20, Words: ["That's", 'That', "'s", 'dangerou-', '–', 'I', 'mean', 's-', '–'], POS Tag true : ['_', 'DT', 'VBZ', 'JJ', ':', 'PRP', 'VBP', 'UH', ':'], POS Tag predicted: ['_', 'DT', 'VBZ', 'JJ', ':', 'PRP', 'VBP', 'UH', ':']
Sentence ID: GUM_conversation_gossip-21, Words: ['Cause', 'he', 'could', 'have', ',', 'brain', 'damage', '.'], POS Tag true : ['IN', 'PRP', 'MD', 'VB', ',', 'NN', 'NN', '.'], POS Tag predicted: ['IN', 'PRP', 'MD', 'VB', ',', 'NN', 'NN', '.']
Sentence ID: GUM_conversation_gossip-22, Words: ['The', 'doctor', 'said', '–', 'the', 'do

In [ ]:
print(len(POS_tags_train_true))
print(len(POS_tags_train_pred))

36712
36712


In [ ]:
df_output_train = pd.DataFrame(output_train)
tsv_file_path = 'viterbi_predictions_train.tsv'

# Write the DataFrame to the TSV file
df_output_train.to_csv(tsv_file_path, sep='\t', index=False)
print("TSV file has been created successfully.")

TSV file has been created successfully.


In [ ]:
for true_tag, predicted_tag in zip(POS_tags_train_true, POS_tags_train_pred):
    print(f"True Tag: {true_tag},  Predicted Tag: {predicted_tag}")

Streaming output truncated to the last 5000 lines.
True Tag: UH,  Predicted Tag: UH
True Tag: .,  Predicted Tag: .
True Tag: PRP,  Predicted Tag: PRP
True Tag: VBP,  Predicted Tag: VBP
True Tag: _,  Predicted Tag: _
True Tag: PRP,  Predicted Tag: PRP
True Tag: MD,  Predicted Tag: MD
True Tag: VB,  Predicted Tag: VB
True Tag: TO,  Predicted Tag: TO
True Tag: VB,  Predicted Tag: VB
True Tag: IN,  Predicted Tag: IN
True Tag: DT,  Predicted Tag: DT
True Tag: NN,  Predicted Tag: NN
True Tag: .,  Predicted Tag: .
True Tag: RB,  Predicted Tag: RB
True Tag: .,  Predicted Tag: .
True Tag: VBD,  Predicted Tag: VBD
True Tag: DT,  Predicted Tag: DT
True Tag: JJ,  Predicted Tag: JJ
True Tag: NN,  Predicted Tag: NN
True Tag: .,  Predicted Tag: .
True Tag: UH,  Predicted Tag: UH
True Tag: .,  Predicted Tag: .
True Tag: UH,  Predicted Tag: UH
True Tag: .,  Predicted Tag: .
True Tag: VBP,  Predicted Tag: VBP
True Tag: PDT,  Predicted Tag: PDT
True Tag: DT,  Predicted Tag: DT
True Tag: NN,  Predicted Ta

In [ ]:
# Calculate precision, recall, F-score, and accuracy
precision_train = precision_score(POS_tags_train_true, POS_tags_train_pred, average='micro')
recall_train = recall_score(POS_tags_train_true, POS_tags_train_pred, average='micro')
f1_score_train = f1_score(POS_tags_train_true, POS_tags_train_pred, average='micro')
accuracy_train = accuracy_score(POS_tags_train_true, POS_tags_train_pred)

# Print the results
print("Precision Score:", precision_train)
print("Recall Score:", recall_train)
print("F-score Score:", f1_score_train)
print("Accuracy Score:", accuracy_train)

Precision Score: 0.9807964698191327
Recall Score: 0.9807964698191327
F-score Score: 0.9807964698191327
Accuracy Score: 0.9807964698191327


# Test

Function to calculate emission probabilities from tag(t) to word(w) using Add-1-Smoothing

P(W | t) = (Count(t, w) +  1) / (Count(t) + |V|)

where |V| is the size of the vocabulary

In [ ]:
def emission_prob_Add1_smoothing(w,t):
  if (w,t) in tag_word_pair_counts:
      return (tag_word_pair_counts[(w,t)] + 1 )/( tag_freq[t]+ len(word_freq))
  else :
      return 1/(tag_freq[t]+ len(word_freq))

Viterbi for test with Add-1-smoothing for emission probabilities

In [ ]:
def viterbi_test(sentence, tags_set):
    T=len(tags_set)
    W= len(sentence)
    epsilon = 1

    # Initialize the dynamic programming table to scores and tags found at each step
    scores= np.zeros((T,W), dtype= float)
    backptr = np.zeros((T,W), dtype=int)

    # Initialization Step for w==0 i,e. 1st word of sentence
    for t in range(T):
          scores[t,0]= (tag_start_prob(tags_set[t])* emission_prob_Add1_smoothing(sentence[0], tags_set[t]))
          backptr[t, 0] = 0

    # Iteration Step
    for w in range (1,W):
      for t in range(T):
        prob= scores[:, w-1] * transition_matrix[:, t] * emission_prob_Add1_smoothing(sentence[w], tags_set[t])

        backptr[t, w] = np.argmax(prob)
        scores[t, w] = np.max(prob)

    best_path_prob = np.max(scores[:, -1])

    # Path backtracking
    best_path = np.zeros(len(sentence), dtype=int)
    best_path[-1] = np.argmax(scores[:, -1])
    for w in range(len(sentence)-2, -1, -1):
        best_path[w] = backptr[best_path[w+1], w+1]

    POS_tag_seq = [tags_set[index] for index in best_path]
    return POS_tag_seq

In [ ]:
# Initialize lists to store output of test
output_test = {
    'sent_id': [],
    'token_no': [],
    'word': [],
    'predicted_tag': []
}

POS_tags_test_pred=[]
POS_tags_test_true=[]
for sent_id, group in tqdm(df_test.groupby('sent_id')):
  sentence = group['word'].tolist()
  true_tag = group['POS_tag'].to_list()
  pred_tag = viterbi_test(sentence, tags_set)
  POS_tags_test_pred.extend(pred_tag)
  POS_tags_test_true.extend(true_tag)
  print(f"Sentence ID: {sent_id}, Words: {sentence}, POS Tag true : {true_tag}, POS Tag predicted: {pred_tag}")

  # Store the test output for each word in the sentence
  for i, (word, true_tag, pred_tag) in enumerate(zip(sentence, true_tag, pred_tag), start=1):
      output_test['sent_id'].append(sent_id)
      output_test['token_no'].append(i)
      output_test['word'].append(word)
      output_test['predicted_tag'].append(pred_tag)

  3%|▎         | 3/100 [00:00<00:04, 22.60it/s]

Sentence ID: GUM_academic_discrimination-1, Words: ['The', 'prevalence', 'of', 'discrimination', 'across', 'racial', 'groups', 'in', 'contemporary', 'America', ':'], POS Tag true : ['DT', 'NN', 'IN', 'NN', 'IN', 'JJ', 'NNS', 'IN', 'JJ', 'NNP', ':'], POS Tag predicted: ['DT', 'NN', 'IN', 'NN', 'IN', 'JJ', 'NNS', 'IN', 'JJ', 'NNS', ':']
Sentence ID: GUM_academic_discrimination-10, Words: ['What', 'also', 'should', 'not', 'be', 'lost', 'in', 'discussion', 'of', 'discrimination', 'is', 'the', 'growing', 'push', 'to', 'implement', 'social', 'policy', 'aimed', 'at', 'reducing', 'the', 'occurrence', 'of', 'discriminatory', 'practices', '.'], POS Tag true : ['WP', 'RB', 'MD', 'RB', 'VB', 'VBN', 'IN', 'NN', 'IN', 'NN', 'VBZ', 'DT', 'VBG', 'NN', 'TO', 'VB', 'JJ', 'NN', 'VBN', 'IN', 'VBG', 'DT', 'NN', 'IN', 'JJ', 'NNS', '.'], POS Tag predicted: ['_', 'PRP', 'MD', 'RB', 'VB', 'VBN', 'IN', 'NN', 'IN', 'DT', 'VBZ', 'DT', 'JJ', 'NN', 'IN', 'DT', 'JJ', 'NN', 'NN', 'IN', 'VBG', 'DT', 'NN', 'IN', 'DT', 

 10%|█         | 10/100 [00:00<00:03, 23.53it/s]

Sentence ID: GUM_academic_discrimination-14, Words: ['At', 'least', 'one', 'prior', 'study', 'by', 'Kessler', 'and', 'colleagues', '[', '15', ']', ',', 'however', ',', 'using', 'measures', 'of', 'perceived', 'discrimination', 'in', 'a', 'large', 'American', 'sample', ',', 'reported', 'that', 'approximately', '33', '%', 'of', 'respondents', 'reported', 'some', 'form', 'of', 'discrimination', '(', 'see', 'also', ',', 'Gibbons', 'et', 'al.', '[', '4', ']', ')', '.'], POS Tag true : ['IN', 'JJS', 'CD', 'JJ', 'NN', 'IN', 'NNP', 'CC', 'NNS', '-LRB-', 'CD', '-RRB-', ',', 'RB', ',', 'VBG', 'NNS', 'IN', 'VBN', 'NN', 'IN', 'DT', 'JJ', 'JJ', 'NN', ',', 'VBD', 'IN', 'RB', 'CD', 'NN', 'IN', 'NNS', 'VBD', 'DT', 'NN', 'IN', 'NN', '-LRB-', 'VB', 'RB', ',', 'NNP', 'FW', 'FW', '-LRB-', 'CD', '-RRB-', '-RRB-', '.'], POS Tag predicted: ['IN', 'DT', 'NN', 'IN', 'NN', 'IN', 'NN', 'CC', 'NNP', '-LRB-', 'CD', '-RRB-', ',', 'RB', ',', 'VBG', 'NN', 'IN', 'DT', 'NN', 'IN', 'DT', 'JJ', 'JJ', 'NN', ',', 'CC', 'IN'

 28%|██▊       | 28/100 [00:00<00:01, 54.16it/s]

...

Sentence ID: GUM_academic_eegimaa-32, Words: ['Nowadays', 'most', 'Eegimaa', 'speakers', 'from', 'Mof-Ávvi', 'live', 'outside', 'their', 'homeland', ',', 'generally', 'in', 'urban', 'areas', 'like', 'Ziguinchor', 'and', 'Dakar', '.'], POS Tag true : ['RB', 'JJS', 'NNP', 'NNS', 'IN', 'NNP', 'VBP', 'IN', 'PRP$', 'NN', ',', 'RB', 'IN', 'JJ', 'NNS', 'IN', 'NNP', 'CC', 'NNP', '.'], POS Tag predicted: ['PRP', 'VBD', 'DT', 'NN', 'IN', 'DT', 'NN', 'IN', 'PRP$', 'NN', ',', 'RB', 'IN', 'DT', 'NNS', 'IN', 'NN', 'CC', 'NN', '.']
Sentence ID: GUM_academic_eegimaa-33, Words: ['Speakers', 'of', 'Eegimaa', 'use', 'the', 'term', 'Gújjolaay', '(', 'meaning', 'Jóola', ')', 'to', 'refer', 'to', 'their', 'language', 'when', 'they', 'speak', 'to', 'each', 'other', ',', 'but', 'use', 'Eegimaa', 'to', 'distinguish', 'their', 'Jóola', 'variety', 'from', 'that', 'of', 'other', 'Jóola', 'peoples', '.'], POS Tag true : ['NNS', 'IN', 'NNP', 'VBP', 'DT', 'NN', 'NNP', '-LRB-', 'VBG', 'NNP', '-RRB-', 'TO', 'VB', 'IN'

 95%|█████████▌| 95/100 [00:02<00:00, 32.56it/s]

Sentence ID: GUM_academic_eegimaa-7, Words: ['Speakers', 'of', 'these', 'languages', 'are', 'found', 'in', 'the', 'Gambia', ',', 'in', 'the', 'former', 'Casamance', 'region', 'of', 'Southern', 'Senegal', 'and', 'in', 'Guinea', 'Bissau', '.'], POS Tag true : ['NNS', 'IN', 'DT', 'NNS', 'VBP', 'VBN', 'IN', 'DT', 'NNP', ',', 'IN', 'DT', 'JJ', 'NNP', 'NN', 'IN', 'NNP', 'NNP', 'CC', 'IN', 'NNP', 'NNP', '.'], POS Tag predicted: ['RB', 'IN', 'DT', 'NNS', 'VBP', 'VBN', 'IN', 'DT', 'NN', ',', 'IN', 'DT', 'JJ', 'NN', 'NN', 'IN', 'DT', 'NN', 'CC', 'IN', 'DT', 'NN', '.']
Sentence ID: GUM_academic_eegimaa-8, Words: ['The', 'map', 'in', 'Figure', '1', ',', 'taken', 'from', 'Ethnologue', ',', 'presents', 'the', 'languages', 'of', 'Senegal', '.'], POS Tag true : ['DT', 'NN', 'IN', 'NNP', 'CD', ',', 'VBN', 'IN', 'NNP', ',', 'VBZ', 'DT', 'NNS', 'IN', 'NNP', '.'], POS Tag predicted: ['DT', 'NN', 'IN', 'NNP', 'CD', ',', 'VBN', 'IN', 'NNP', ',', 'IN', 'DT', 'NNS', 'IN', 'NN', '.']
Sentence ID: GUM_academic_

100%|██████████| 100/100 [00:02<00:00, 33.82it/s]

Sentence ID: GUM_bio_dvorak-5, Words: ["Dvořák's", 'Dvořák', "'s", 'own', 'style', 'has', 'been', 'described', 'as', '"', 'the', 'fullest', 'recreation', 'of', 'a', 'national', 'idiom', 'with', 'that', 'of', 'the', 'symphonic', 'tradition', ',', 'absorbing', 'folk', 'influences', 'and', 'finding', 'effective', 'ways', 'of', 'using', 'them', '"', '.', '[', '1', ']'], POS Tag true : ['_', 'NNP', 'POS', 'JJ', 'NN', 'VBZ', 'VBN', 'VBN', 'IN', '``', 'DT', 'JJS', 'NN', 'IN', 'DT', 'JJ', 'NN', 'IN', 'DT', 'IN', 'DT', 'JJ', 'NN', ',', 'VBG', 'NN', 'NNS', 'CC', 'VBG', 'JJ', 'NNS', 'IN', 'VBG', 'PRP', "''", '.', '-LRB-', 'CD', '-RRB-'], POS Tag predicted: ['_', 'PRP', 'VBZ', 'JJ', 'NN', 'VBZ', 'VBN', 'VBN', 'IN', '``', 'DT', 'JJ', 'NN', 'IN', 'DT', 'JJ', 'NN', 'IN', 'DT', 'IN', 'DT', 'JJ', 'NN', ',', 'CD', '-RRB-', ',', 'CC', 'DT', 'JJ', 'NNS', 'IN', 'DT', 'NN', "''", '.', '-LRB-', 'CD', '-RRB-']
Sentence ID: GUM_bio_dvorak-6, Words: ['Antonín', 'Dvořák'], POS Tag true : ['NNP', 'NNP'], POS Tag 

In [ ]:
for true_tag, predicted_tag in zip(POS_tags_test_true, POS_tags_test_pred):
    print(f"True Tag: {true_tag},  Predicted Tag: {predicted_tag}")

True Tag: DT,  Predicted Tag: DT
True Tag: NN,  Predicted Tag: NN
True Tag: IN,  Predicted Tag: IN
True Tag: NN,  Predicted Tag: NN
True Tag: IN,  Predicted Tag: IN
True Tag: JJ,  Predicted Tag: JJ
True Tag: NNS,  Predicted Tag: NNS
True Tag: IN,  Predicted Tag: IN
True Tag: JJ,  Predicted Tag: JJ
True Tag: NNP,  Predicted Tag: NNS
True Tag: :,  Predicted Tag: :
True Tag: WP,  Predicted Tag: _
True Tag: RB,  Predicted Tag: PRP
True Tag: MD,  Predicted Tag: MD
True Tag: RB,  Predicted Tag: RB
True Tag: VB,  Predicted Tag: VB
True Tag: VBN,  Predicted Tag: VBN
True Tag: IN,  Predicted Tag: IN
True Tag: NN,  Predicted Tag: NN
True Tag: IN,  Predicted Tag: IN
True Tag: NN,  Predicted Tag: DT
True Tag: VBZ,  Predicted Tag: VBZ
True Tag: DT,  Predicted Tag: DT
True Tag: VBG,  Predicted Tag: JJ
True Tag: NN,  Predicted Tag: NN
True Tag: TO,  Predicted Tag: IN
True Tag: VB,  Predicted Tag: DT
True Tag: JJ,  Predicted Tag: JJ
True Tag: NN,  Predicted Tag: NN
True Tag: VBN,  Predicted Tag: NN
Tr

In [ ]:
# Calculate precision, recall, F-score, and accuracy
precision_test = precision_score(POS_tags_test_true, POS_tags_test_pred, average='micro')
recall_test = recall_score(POS_tags_test_true, POS_tags_test_pred, average='micro')
f1_score_test = f1_score(POS_tags_test_true, POS_tags_test_pred, average='micro')
accuracy_test = accuracy_score(POS_tags_test_true, POS_tags_test_pred)

# Print the results
print("Precision Score:", precision_test)
print("Recall Score:", recall_test)
print("F-score Score:", f1_score_test)
print("Accuracy Score:", accuracy_test)

Precision Score: 0.7106227106227107
Recall Score: 0.7106227106227107
F-score Score: 0.7106227106227107
Accuracy Score: 0.7106227106227107


In [ ]:
df_output_test = pd.DataFrame(output_test)
tsv_file_path = 'viterbi_predictions_test.tsv'

# Write the DataFrame to the TSV file
df_output_test.to_csv(tsv_file_path, sep='\t', index=False)
print("TSV file has been created successfully.")

TSV file has been created successfully.
